In [9]:
import pickle
with open('luby.pickle', 'rb') as f:
    res = pickle.load(f)
res

{'version': 'V2',
 'requestId': 'dc1087b7-0bd6-4986-b038-5d935038e077',
 'timestamp': 1634022712163,
 'images': [{'uid': 'f9008d4df10f405e83dfe6b645717d69',
   'name': 'demo',
   'inferResult': 'SUCCESS',
   'message': 'SUCCESS',
   'validationResult': {'result': 'NO_REQUESTED'},
   'fields': [{'valueType': 'ALL',
     'boundingPoly': {'vertices': [{'x': 518.0, 'y': 284.0},
       {'x': 704.0, 'y': 284.0},
       {'x': 704.0, 'y': 334.0},
       {'x': 518.0, 'y': 334.0}]},
     'inferText': 'LETTER',
     'inferConfidence': 1.0,
     'type': 'NORMAL',
     'lineBreak': False},
    {'valueType': 'ALL',
     'boundingPoly': {'vertices': [{'x': 710.0, 'y': 284.0},
       {'x': 788.0, 'y': 284.0},
       {'x': 788.0, 'y': 334.0},
       {'x': 710.0, 'y': 334.0}]},
     'inferText': 'OF',
     'inferConfidence': 0.9999,
     'type': 'NORMAL',
     'lineBreak': False},
    {'valueType': 'ALL',
     'boundingPoly': {'vertices': [{'x': 794.0, 'y': 284.0},
       {'x': 1099.0, 'y': 284.0},
    

In [2]:
def make_sentence(res):
    # 제이슨 형식의 텍스트 중 inferText와 boundingPoly만 추출하기
    text=res['images'][0]['fields']
    
    # 각각 text와 좌표를 담는 리스트
    infertext=[]
    bounding=[]
    for i in range(0, len(text)):
        infertext.append(text[i].get('inferText'))
        bounding.append(text[i].get('boundingPoly').get('vertices'))
    
    # y 좌표의 평균값을 추출하는 리스트(한줄로 만들기 위해서 Y의 평균을 사용)
    bounding_mean=[]
    # x 좌표를 통해 한 줄을 구분해보자
    x_bounding_max=[]
    x_bounding_min=[]
    for i in range(0, len(bounding)):
        x=[]
        y=[]
        for j in range(0,4):
            y.append(bounding[i][j]['y'])
            x.append(bounding[i][j]['x'])
        bounding_mean.append(sum(y)/len(y))
        x_bounding_max.append(max(x))
        x_bounding_min.append(min(x))
    
    # 텍스트와 y 평균을 한 곳으로 모음
    words = list(zip(infertext,bounding_mean,x_bounding_max,x_bounding_min))
    words

    # y좌표의 평균이 똑같거나 차이가 10 이하인 애들을 한줄로 인식해서 리스트에 담음
    result=[]         # 한 문장을 담는 리스트 
    col=[]
    mean=[]           # 문장의 y값(각 텍스트의 y평균들의 평균)을 담은 리스트
    exp=[]
    
    i=0
    while i<len(words):
        try:
            # (바로 앞뒤 텍스트의 y좌표 평균이 같거나 10보다 작을경우) 그리고 앞뒤 텍스트의 거리가 20보다 작은 경우
            if abs(words[i][1]-words[i+1][1])<=10 and abs(words[i][2]-words[i+1][3])<=20:
                col.append(words[i][0])
            else:
                col.append(words[i][0])
                result.append(' '.join(col))
                for word in col:
                    exp.append(bounding_mean[infertext.index(word)])
                mean.append(sum(exp)/len(exp))
                exp=[]
                col=[]
        except: # 맨 뒤에 단어는 비교해줄 단어가 없기 때문에!
            col.append(words[i][0])
            result.append(' '.join(col))
            for word in col:
                exp.append(bounding_mean[infertext.index(word)])
            mean.append(sum(exp)/len(exp))
            exp=[]
            col=[]
        i+=1
        
    return result

- sanction list 가져올 때, 일단 sdn 파일에서 가져왔어

In [3]:
with open('sdn.pickle', 'rb') as f:
    sdn_list = pickle.load(f)
sdn_list

['ANGLO-CARIBBEAN CO., LTD.',
 'BANCO NACIONAL DE CUBA',
 'BOUTIQUE LA MAISON',
 'CASA DE CUBA',
 'CECOEX, S.A.',
 'CIMEX',
 'CIMEX IBERICA',
 'CIMEX, S.A.',
 'COMERCIAL IBEROAMERICANA, S.A.',
 'COMERCIAL CIMEX, S.A.',
 'COMERCIAL DE RODAJES Y MAQUINARIA, S.A.',
 'COMERCIALIZACION DE PRODUCTOS VARIOS',
 'COMPANIA DE IMPORTACION Y EXPORTACION IBERIA',
 'CORPORACION CIMEX, S.A.',
 'COTEI',
 'CRYMSA - ARGENTINA, S.A.',
 'CUBACANCUN CIGARS AND GIFT SHOPS',
 'CUBAEXPORT',
 'CUBAFRUTAS',
 'CUBAN CIGARS TRADE',
 'CUBANATUR',
 'CUBATABACO',
 'CUMEXINT, S.A.',
 'DELVEST HOLDING, S.A.',
 'EDICIONES CUBANAS',
 'EDYJU, S.A.',
 'EMPRESA CUBANA DE AVIACION',
 'EMPRESA CUBANA DE PESCADOS Y MARISCOS',
 'EMPRESA DE TURISMO NACIONAL Y INTERNACIONAL',
 'ETCO INTERNATIONAL COMMODITIES LTD.',
 'ETCO INTERNATIONAL COMPANY, LIMITED',
 'GALAX TRADING CO., LTD.',
 'HAVIN BANK LIMITED',
 'HAVANATUR',
 'INSTITUTO NACIONAL DE TURISMO DE CUBA',
 'INTERCONSULT',
 'KAVE, S.A.',
 'LA EMPRESA CUBANA DE FLETES',
 'LEVE

In [4]:
def text_similarity(result):
    import textdistance
    sanction=[]
    for i in result:
        for j in sdn_list:
            if textdistance.damerau_levenshtein(i,j) < 2:
                sanction.append(i)
    return sanction

In [5]:
real_sanction=text_similarity(make_sentence(res))

In [7]:
real_sanction

['ALCALDE LINARES, Angel']

NameError: name 'a' is not defined

In [ ]:
vertices=[]
raw_text=[]
for i in range(len(res['images'][0]['fields'])):
    vertices.append(res['images'][0]['fields'][i]['boundingPoly']['vertices'])
    raw_text.append(res['images'][0]['fields'][i]['inferText'])
#     vertices.append(v[i]['boundingPoly']['vertices'])
#     raw_test.append(v[i]['inferText'])
    

In [ ]:
willuse=list(zip(raw_text,vertices))
willuse

In [ ]:
while len(real_sanction)!=0:
    real_sanction[0]

In [ ]:
real_sanction[0][0]

In [ ]:
text = res['images'][0]['fields']
infertext=[]
bounding=[]

for i in range(0, len(text)):
    infertext.append(text[i].get('inferText'))
    bounding.append(text[i].get('boundingPoly').get('vertices'))

bounding_mean=[]
x_bounding_max=[]
x_bounding_min=[]
all_x=[] 
all_y=[]
for i in range(0, len(bounding)):
    x=[]
    y=[]
    for j in range(0,4):
        y.append(bounding[i][j]['y'])
        x.append(bounding[i][j]['x'])
    bounding_mean.append(sum(y)/len(y))
    x_bounding_max.append(max(x))
    x_bounding_min.append(min(x))
    all_x.append(x)
    all_y.append(y)

words = list(zip(infertext,bounding_mean,x_bounding_max,x_bounding_min,list(range(len(infertext)))))
words

In [ ]:
all_x

In [ ]:
all_y

# 재영이한테 넘겨주는 파일

In [10]:
with open('jaeyoung.pickle', "rb") as fh:
  rect = pickle.load(fh)
rect

[{'boundingPoly': {'vertices': [[{'x': 0, 'y': 0},
     {'x': 1661, 'y': 0},
     {'x': 1661, 'y': 2337},
     {'x': 0, 'y': 2337}]]},
  'uuid21': 'BvD'},
 {'boundingPoly': {'vertices': [[{'x': 47, 'y': 30},
     {'x': 1630, 'y': 30},
     {'x': 1630, 'y': 47},
     {'x': 47, 'y': 47}]]},
  'uuid24': 'a'},
 {'boundingPoly': {'vertices': [[{'x': 47, 'y': 30},
     {'x': 1630, 'y': 30},
     {'x': 1630, 'y': 47},
     {'x': 47, 'y': 47}]]},
  'uuid26': 'es'},
 {'boundingPoly': {'vertices': [[{'x': 47, 'y': 30},
     {'x': 1630, 'y': 30},
     {'x': 1630, 'y': 47},
     {'x': 47, 'y': 47}]]},
  'uuid34': 'Conetyices'},
 {'boundingPoly': {'vertices': [[{'x': 47, 'y': 30},
     {'x': 1630, 'y': 30},
     {'x': 1630, 'y': 47},
     {'x': 47, 'y': 47}]]},
  'uuid35': 'Aileoal'},
 {'boundingPoly': {'vertices': [[{'x': 313, 'y': 42},
     {'x': 1630, 'y': 42},
     {'x': 1630, 'y': 52},
     {'x': 313, 'y': 52}]]},
  'uuid36': 'Ninbar'},
 {'boundingPoly': {'vertices': [[{'x': 313, 'y': 42},
   

In [11]:
def make_sentence(rect):    
    # 각각 text와 좌표를 담는 리스트
    infertext=[]
    bounding=[]
    uuid=[]
    for i in range(0, len(rect)):
        infertext.append(rect[i].get(list(rect[i].keys())[-1]))
        bounding.append(rect[i]['boundingPoly']['vertices'][0])
        uuid.append(list(rect[i].keys())[-1])
    
    # y 좌표의 평균값을 추출하는 리스트(한줄로 만들기 위해서 Y의 평균을 사용)
    bounding_mean=[]
    # x 좌표를 통해 한 줄을 구분해보자
    x_bounding_max=[]
    x_bounding_min=[]
    for i in range(0, len(bounding)):
        x=[]
        y=[]
        for j in range(0,4):
            y.append(bounding[i][j]['y'])
            x.append(bounding[i][j]['x'])
        bounding_mean.append(sum(y)/len(y))
        x_bounding_max.append(max(x))
        x_bounding_min.append(min(x))
    
    # 텍스트와 y 평균을 한 곳으로 모음
    words = list(zip(infertext,bounding_mean,x_bounding_max,x_bounding_min))
    words

    # y좌표의 평균이 똑같거나 차이가 10 이하인 애들을 한줄로 인식해서 리스트에 담음
    result=[]         # 한 문장을 담는 리스트 
    col=[]
    mean=[]           # 문장의 y값(각 텍스트의 y평균들의 평균)을 담은 리스트
    exp=[]
    
    i=0
    while i<len(words):
        try:
            # (바로 앞뒤 텍스트의 y좌표 평균이 같거나 10보다 작을경우) 그리고 앞뒤 텍스트의 거리가 20보다 작은 경우
            if abs(words[i][1]-words[i+1][1])<=30 and abs(words[i][2]-words[i+1][3])<=20:
                col.append(words[i][0])
            else:
                col.append(words[i][0])
                result.append(' '.join(col))
                for word in col:
                    exp.append(bounding_mean[infertext.index(word)])
                mean.append(sum(exp)/len(exp))
                exp=[]
                col=[]
        except: # 맨 뒤에 단어는 비교해줄 단어가 없기 때문에!
            col.append(words[i][0])
            result.append(' '.join(col))
            for word in col:
                exp.append(bounding_mean[infertext.index(word)])
            mean.append(sum(exp)/len(exp))
            exp=[]
            col=[]
        i+=1
        
    return result

In [12]:
make_sentence(rect)

['BvD',
 'a',
 'es',
 'Conetyices',
 'Aileoal',
 'Ninbar',
 'BP',
 'hate',
 'Ts',
 'mage',
 'EAA',
 'HAIL',
 'TOTS',
 'TiO SPRATT ny ae',
 'A',
 'BE',
 'CARRIED',
 'BeORT ee',
 'SrSenaneGr fir',
 'ra',
 'Ss',
 'RULIEHOJOHE',
 'Set',
 'sooner',
 'CONTAAY',
 'STRLSETIONS',
 're',
 'AD',
 'Srrees',
 'a CARI APPROP',
 'i',
 'HL',
 'L NY',
 'BRAIN',
 'fon',
 'Te',
 'NOWOE CONCERNING CARRIERS LIMITATION LABILITY Shipper',
 'may',
 'x ro increase such Aerie lon',
 'of',
 'Zab',
 'higher vetue for cari',
 'fe LEE AE',
 'i',
 'loess',
 'auch',
 'ky',
 'by',
 'dectaring a',
 'high',
 'age',
 'aap',
 'spe',
 'Aimeit',
 'of',
 'Departuce',
 'Adcrass',
 'of',
 'firet Carierfand Raquesiea',
 'Fiouting HYDERABAD AIRPORT WSURANCE If Cartier offers jasurance ond',
 'such',
 'insurance',
 'is',
 'Foquosted',
 'in accoranca with',
 'the',
 'condllicea',
 'tnerest',
 'Indicate amount to by Re P',
 'B',
 'be insured in figures',
 'in',
 'box',
 'markad',
 'Amourt af Insurance',
 'Fe Tandy',
 'Information s

In [16]:
def text_similarity(result):
    import textdistance
    sanction=[]
    for i in result:
        for j in sdn_list:
            if textdistance.jaro_winkler(i,j) > 0.9:
                sanction.append(i)
    return sanction

In [17]:
jaeyoung_sanction=text_similarity(make_sentence(rect))

In [18]:
def text_similarity_jaro(result):
    import textdistance
    sanction=[]
    return [i for i in result for j in sdn_list if textdistance.jaro_winkler(i,j) > 0.9]

In [21]:
jaeyoung_sanction

['CARE']